In [284]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from IES_Downloader import IES_Downloader
import pandas as pd
import math
import operator


In [283]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [137]:
pip install scrapy

  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp37-none-any.whl size=12551 sha256=3c0fb2aaed7fef903e0e99ef4b015da27be1b1b26ec4e6facc11067811d42038
  Stored in directory: C:\Users\marketa.cervena\AppData\Local\pip\Cache\wheels\88\99\96\cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp37-none-any.whl size=7770 sha256=d65e1c051f09abea3953592b824be3eeb74fa4945ea18f1473ae281614416d6e
  Stored in directory: C:\Users\marketa.cervena\AppData\Local\pip\Cache\wheels\51\01\d1\4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatcher protego
Note: you may need to restart the kernel to use updated packages.


In [267]:
links = ("https://www.czc.cz/mobilni-telefony","https://www.datart.cz/mobilni-telefony","https://www.euronics.cz/mobilni-telefony", "https://www.mall.cz/https://www.mall.cz/mobilni-telefony")

shopnames = {
  links[0]: "CZC",
  links[1]: "DATART",
  links[2]: "EURONICS",
 links[3]: "MALL"
}


In [293]:
class eshop:
    '''
   The main class containing all infomation gathered from eshop page, including the information that 
   requires web-specific scraping methods. As a "bonus", it also backs up the soup objects of all product pages in a SQL database
    '''
    def __init__(self, link, category = "notebooky"):
        self.shop_name = link.split(".")[1].split("/")[0]
            
        if self.shop_name == "czc": # if we scrape CZC eshop, use the link and categroy to create object 
            #from the specific CZC class that has its own scraping methods defined bellow
            
            self.shop_obj = CZC(link, category)
            
        elif self.shop_name == "datart": #similar for Datart eshop
            
            self.shop_obj == DATART(link, category)
        
        else:
            print("Eshop {} out of scope".format(self.shop_name))
            self.shop_obj == []
            
        try: 
            self.robots_link = link + "/robots.txt"
            self.robots = requests.get(self.robots_link).text
        except: 
            print("Cannot reach robots.txt sie")
            
        self.cat_link = [] #link to to the first product page within category
        self.itms = [] #number of items in category
        self.pgs = [] # number of product pages in cateogory
        self.itmppg = [] #items per page
        self.urls = [] #list of URLs to all product pages within category
        self.allsoups = [] #list of BS objects for the urls list
        self.all_pages = [] #list of page objects from the BSs
        self.prd_modules = [] #list of product modules found in the product page BSs
        self.all_prds = []#list of product objects
        self.prd_data = {
                'product':[],
                'price':[]} 
        

    def get_catlink(self):
        '''
        formulation of the main product page link requires eshop specific method -
        '''    
        self.shop_obj.get_catlink()
        self.cat_link = self.shop_obj.cat_link
        
    
    def get_homesoup(self):#generic method
        '''
        simplegeneric  request method 
        '''     
        #try: 
        r = requests.get(self.cat_link)
        r.encoding='UTF-8'
        self.home_soup = BeautifulSoup(r.text,'lxml')
            
        #some expect here "Could not request the eshop main page"
        
    def leverage_soup(self): 
        '''
        gathering initial information from the first product page via self.shop_obj 
        '''  
        self.shop_obj.get_homesoupinfo(self.home_soup)
        self.pgs = self.shop_obj.pgs
        self.itms = self.shop_obj.itms
        self.itmppg = math.ceil(self.itms / self.pgs)
             
        self.urls = [None]*self.pgs
        
        self.urls[0] = self.cat_link
        
        for i in range(1,self.pgs):
            self.urls[i] = self.cat_link + self.shop_obj.urlspec + str(self.itmppg*i)
    
    def get_allsoups(self):
        '''
        lists soup objects of all product pages within the category
        '''  
        self.allsoups = [None]*self.pgs
        mm = ""
        
        for i in range(0,self.pgs):
            #try:
            r = requests.get(self.urls[i])
            r.encoding='UTF-8'
            self.allsoups[i] = BeautifulSoup(r.text,'lxml')
                #some exceot 
                #m = "Could not request {}th product page from {} eshop".format(i,self.shop_name)
                #mm = appent(mm,m)
                #print(mm)
            
            
    def get_pages(self):
        '''
        creates eshop-specific list of page objects 
        '''  
        self.pages = self.shop_obj.get_pages(self.allsoups)
        
        for i in range (0,self.pgs):
            self.prd_modules[((0+i)*self.itmppg):((1+i)*self.itmppg)] = self.pages[i].prodmod
        
        
    def get_products(self):
        '''
        Extrect the relevant information from the identified product modulesusingehop-specific methods
        '''
        
        
        for i in range (0,self.itms):
            self.all_prds[i] = product(self.shop_object, self.prd_modules[i]).GetTitle().GetPrice()
            self.prd_data['product'][i] = self.all_prds[i].title
            self.prd_data['price'][i] = self.all_prds[i].price
            
        
    def get_output(self):
        '''
       Formulate the collected information in a panda frame
        '''
        

    def update(self):
        '''
       Update database
        '''


    
              
            

In [294]:

class DATART:
    '''
    Specific clas for DATART eshop containing its web specific scraping methods under alligned terminology in
    order to be callable in the eshop class 
    '''
    
    def __init__(self, link, category):
        '''
        Define the empty generic eshop structure including the items that require web specific scraping  
        '''
        self.link = link
        self.category = category
        self.cat_link = []
        self.itms = []
        self.pgs = []
        self.urlspec = "/?startPos="
        self.pages = []
        self.products = []
        
 
        
        
    def get_catlink(self):
        '''
        Contruct the actual link for relevant category in DATART eshop  
        '''    
        self.cat_link = "{}/{}".format(self.link, self.category)
        
    def get_homesoupinfo(self, soup):
        '''
        Download the beautiful soup object from the first product page in selected category on DATART and extract available information
        '''    
        self.itms = int(soup.findAll("div",{'class','pagination-show-page'})[0].findAll('strong')[1].text)
        self.pgs =  int(soup.findAll("a",{'class','button filter-attr'})[2].text)
        
    def get_pages(self, souplist):
        
        self.pages = [None]*self.pgs
        for i in range(0,len(souplist)):
            self.pages[i] = page(souplist[i]).get_prodsoups()
    
        
        
        
         
    
class page(DATART):
        
    def __init__(self, soup):
        '''
        Define the empty generic eshop structure including the items that require web specific scraping  
        '''
        self.soup = soup
        self.prodmod = []
        self.products = []
    
    def get_prodsoups(self):
        '''
        Extracts the product modules pro soup of all pages within the category
        '''
        self.prodmod = self.soup.findAll('div', {'class':'category-page-item grid fly-parent wa-product-impression'})
        get_products(self.prodmod)
        
    def get_products(self, souplist):
        
        self.products = [None]*self.itms
        for i in range(0,len(souplist)):
            self.products[i] = product(souplist[i]).get_title().get_price()
    
class product(DATART):
    
    def __init__(self, soup):
        '''
        Define the empty generic eshop structure including the items that require web specific scraping  
        '''
        self.soup = soup
        self.title = []
        self.price = []
        self.currency = "CZK"
    
    
    def get_title(self):
        '''
        A method to extract product name from product module soup 
        '''
        self.title = this_soup.find('h3',{'class':'item-title'}).text.replace(u'\r', u'').replace(u'\n', u'').split(" (")[0]
     
    def get_price(self): 
        '''
        A method to extract product price from product module soup 
        '''
        self.price =  int(this_soup.find('span',{'class':'actual'}).text.split(":")[1].replace(u'\xa0', u'').split("Kč")[0])
           
    

    
            

In [295]:
         
        
        
class CZC:
    '''
    Specific clas for CZC eshop containing its web specific scraping methods under alligned terminology in
    order to be callable in the eshop class 
    '''
    
    def __init__(self, link, category):
        '''
        Define the empty generic eshop structure including the items that require web specific scraping  
        '''
        
        self.link = link
        self.category = category
        self.cat_link = []
        self.itms = []
        self.pgs = []
        self.urlspec = "?q-first="
        self.pages = []
        self.products = []
            
        
    def get_catlink(self):
        '''
        Contruct the actual link for relevant category in CZC eshop  
        '''
        self.cat_link = "{}/{}/produkty".format(self.link, self.category)
    
    def get_homesoupinfo(self,soup):
        '''
        Download the beautiful soup object from the first product page in selected category on CZC and extract available information
        '''    
        self.itms = int(str(soup.findAll('div',{'class','order-by-sum h-800'})).split(">")[1].split("<")[0].split(" ")[0].replace(u'\xa0', u''))
        self.pgs = math.ceil(self.itms/27)
        
    def get_pages(self, souplist):
        
        self.pages = [None]*self.pgs
        for i in range(0,len(souplist)):
            self.pages[i] = page(souplist[i]).get_prodsoups() 
            
class page(CZC):
        
    def __init__(self, soup):
        '''
        Define the empty generic eshop structure including the items that require web specific scraping  
        '''
        self.soup = soup
        self.prodmod = []
        self.products = []
    
    def get_prodsoups(self):
        '''
        Extracts the product modules pro soup of all pages within the category
        '''
        self.prodmod = self.soup.findAll('div', {'class':'overflow'})
        get_products(self.prodmod)
        
    def get_products(self, souplist):
        
        self.products = [None]*self.itms
        for i in range(0,len(souplist)):
            self.products[i] = product(souplist[i]).get_title().get_price()
        
class product(CZC):
        
    def __init__(self, soup):
        '''
        Define the empty generic eshop structure including the items that require web specific scraping  
        '''
        self.soup = soup
        self.title = []
        self.price = []
        self.currency = "CZK"
    
    
    def GetTitle(self):
        '''
        A method to extract product name from product module soup 
        '''
        self.title = self.soup.find('a')['title'].split(",")[0].split(" +")[0]
     
    def GetPrice(self): 
        '''
        A method to extract product price from product module soup 
        '''
        price =self.soup.findAll('span',{'class':'price-vatin'})
        l=len(price)-1
        self.price = int(price[l].text.replace(u'\xa0', u'').split("Kč")[0])

In [296]:
R = eshop("https://www.czc.cz","notebooky")
R.get_catlink()
#R.get_homesoup()
#R.leverage_soup()
#R.get_allsoups()
#R.get_pages()


In [278]:
R.shop_name
R.cat_link

'czc'